***
# <font color=red>Chapter 7: MedTALN inc.'s Case Study - Deploy Healthcare NER Model</font>
<p style="margin-left:10%; margin-right:10%;">by <font color=teal> John Doe (typica.ai) </font></p>

***


## Overview

This notebook guides you through the process of deploying our Healthcare NER model. The development of this notebook was based on the following resources:

- [ADS v2.10.0 documentation for the class `HuggingFacePipelineModel`](https://accelerated-data-science.readthedocs.io/en/v2.10.0/user_guide/model_registration/frameworks/huggingfacemodel.html)
- **Train, Register, and Deploy HuggingFace Pipeline Example Notebook** from Notebook Explorer (`train-register-deploy-huggingface-pipeline.ipynb`)


## Process Overview

The deployment process is composed of the following key steps:

1. **Initializing the ADS Class `HuggingFacePipelineModel`**  
    1.1. Load the Hugging Face pipeline with our best fine-tuned Healthcare NER Model (i.e., `healthcare_ner_model_v1.0.0`)  
    1.2. Initialize the `HuggingFacePipelineModel` class with our model  
    1.3. Generate the model’s artifacts

2. **Saving the model’s artifacts to the OCI Data Science Model Catalog**

3. **Creating the OCI Data Science Model Deployment** (with specified compute shape)

Filters out warnings

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Authenticate

Authentication to the OCI Data Science service is required. Here we default to resource principals.

In [2]:
import ads

ads.set_auth(auth="resource_principal")

# Initializing the ADS Class `HuggingFacePipelineModel`

## Initialize Hugging Face Pipeline 
Initialize `transformers.pipeline` with our best fine-tuned Healthcare NER Model (i.e., `healthcare_ner_model_v1.0.0`)  


In [3]:
from transformers import pipeline
import warnings

model_checkpoint = "/home/datascience/buckets/models-ckpt-bkt/models/healthcare_ner_model_v1.0.0" 
print(f"model checkpoint: {model_checkpoint}")

data = "Le medecin donne des antibiotiques en cas d'infections des voies respiratoires."
pipeline = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="first"
)
preds = pipeline(data)
preds

model checkpoint: /home/datascience/buckets/models-ckpt-bkt/models/healthcare_ner_model_v1.0.0


[{'entity_group': 'MedicationVaccine',
  'score': 0.7428154,
  'word': 'antibiotiques',
  'start': 21,
  'end': 34}]

## Prepare Model Artifact
Instantiate a HuggingFacePipelineModel() object with HuggingFace pipelines. All the pipelines related files are saved under the artifact_dir.

In [4]:
from ads.model import HuggingFacePipelineModel
from ads.model.model_metadata import UseCaseType
import tempfile

# Create a temporary directory for the model artifacts
artifacts_temp_dir = tempfile.mkdtemp()

print(f"Model path {pipeline.model.config._name_or_path} and the Model artifacts temp dir {artifacts_temp_dir}")

# Initialize the model
huggingface_pipeline_model = HuggingFacePipelineModel(pipeline, artifact_dir=artifacts_temp_dir)

#Prepare the model
conda_env_source = "oci://conda-envs-bkt@yz2wwgkgt8eh/conda_environments/gpu/PyTorch 2.1 for GPU on Python 3.9/1.0/pytorch21_p39_gpu_v1"

huggingface_pipeline_model.prepare(
  inference_conda_env=conda_env_source,
  inference_python_version="3.9",
  training_conda_env=conda_env_source,
  use_case_type=UseCaseType.OTHER,
  force_overwrite=True,
  )


Model path /home/datascience/buckets/models-ckpt-bkt/models/healthcare_ner_model_v1.0.0 and the Model artifacts temp dir /tmp/tmphplf40sh
[2024-08-30 01:28:25,140] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)                                                                       ?, ?it/s]


algorithm: TokenClassificationPipeline
artifact_dir:
  /tmp/tmphplf40sh:
  - - .model-ignore
    - special_tokens_map.json
    - tokenizer.json
    - config.json
    - runtime.yaml
    - score.py
    - model.safetensors
    - sentencepiece.bpe.model
    - tokenizer_config.json
framework: transformers
model_deployment_id: null
model_id: null

#### Manually correct score.py

- Go to terminal and open score.py e.g. (base) bash-4.2$ vim /tmp/tmpm81d8fz4/score.py
- Add the following modifications

1- In the method load_model, add a correction that introduces an aggregation_strategy parameter for pipeline, specifically set to "first"
comment out the original line:
```
        # model = pipeline(task, model = model_dir)
```
Add the new lines below the commented line:
```
        #John's fix to add ner aggregation_strategy
        model = pipeline(task, model = model_dir, aggregation_strategy="first")
```
2- In the method serialize_prediction, add a correction that fixes the error "The inference result is not json parsable Object of type float32 is not JSON serializable"
```   
    #John's fix for the error "The inference result is not json parsable Object of type float32 is not JSON serializable" 
    if isinstance(yhat, dict):
        for key, value in yhat.items():
            if isinstance(value, np.float32):
                yhat[key] = float(value)
```


In [7]:
import shutil

# Specify the path to the source file (score_fixed.py) and the target location (/tmp/score.py)
source_file = './score_fixed.py'  # Update with the actual path to score_fixed.py
target_file = f'{artifacts_temp_dir}/score.py'

# Copy the source file to the target location, effectively replacing it
shutil.copyfile(source_file, target_file)


'/tmp/tmphplf40sh/score.py'

In [8]:
huggingface_pipeline_model.introspect()

['.model-ignore', 'special_tokens_map.json', 'tokenizer.json', 'config.json', 'runtime.yaml', 'score.py', 'model.safetensors', 'sentencepiece.bpe.model', 'tokenizer_config.json']


,Test key,Test name,Result,Message
0,runtime_env_path,Check that field MODEL_DEPLOYMENT.INFERENCE_ENV_PATH is set,Passed,
1,runtime_env_python,Check that field MODEL_DEPLOYMENT.INFERENCE_PYTHON_VERSION is set to a value of 3.6 or higher,Passed,
2,runtime_path_exist,Check that the file path in MODEL_DEPLOYMENT.INFERENCE_ENV_PATH is correct.,Passed,
3,runtime_version,Check that field MODEL_ARTIFACT_VERSION is set to 3.0,Passed,
4,runtime_yaml,"Check that the file ""runtime.yaml"" exists and is in the top level directory of the artifact directory",Passed,
5,score_load_model,Check that load_model() is defined,Passed,
6,score_predict,Check that predict() is defined,Passed,
7,score_predict_arg,Check that all other arguments in predict() are optional and have default values,Passed,
8,score_predict_data,"Check that the only required argument for predict() is named ""data""",Passed,
9,score_py,"Check that the file ""score.py"" exists and is in the top level directory of the artifact directory",Passed,


## Call model summary
The .summary_status() method returns a Pandas dataframe that guides you through the entire workflow. It shows which methods are available to call and which ones aren’t. Plus it outlines what each method does. If extra actions are required, it also shows those actions.

In [9]:
huggingface_pipeline_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Available     Local tested .predict from score.py                               
save()    Available     Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  UNKNOWN       Deployed the model                                                
predict() Not Available Called deployment predict endpoint

## Verify the generated model artifacts 
without deploying the model to model catalog

In [10]:
print(data)
huggingface_pipeline_model.verify(data)

Le medecin donne des antibiotiques en cas d'infections des voies respiratoires.
Model is successfully loaded.


{'prediction': [{'entity_group': 'MedicationVaccine',
   'score': 0.742815375328064,
   'word': 'antibiotiques',
   'start': 21,
   'end': 34}]}

## Create a model version set
The Model Version Set, which acts as a container by assigning sequential version numbers to models, making it easier to track their evolution and relationships.

In [26]:
from ads.model import ModelVersionSet

# Create a model version set
mvs = ModelVersionSet(
    name = "healthcare-ner-model-ver-set",
    description = "A model version set for the Healthcare NER Model")
mvs.create()


kind: modelVersionSet
spec:
  compartmentId: ocid1.compartment.oc1..aaaaaaaaceavj5r6agl5e2mysyxg6twnvwh6cw7s2pi6nobiv6nynjcwmhxa
  definedTags:
    Oracle-Tags:
      CreatedBy: ocid1.datasciencenotebooksession.oc1.ca-toronto-1.amaaaaaa3hvgr2qaresmmqmzbu3i3mo6npudc3nf6ltgu43in4su7ogons4a
      CreatedOn: '2024-08-30T01:39:55.445Z'
  description: A model version set for the Healthcare NER Model
  id: ocid1.datasciencemodelversionset.oc1.ca-toronto-1.amaaaaaa3hvgr2qa4gck2mvzy6r5nbaumfmdbv7rsilu6x5dnggvg5kxrhxa
  name: healthcare-ner-model-ver-set
  projectId: ocid1.datascienceproject.oc1.ca-toronto-1.amaaaaaa3hvgr2qaqd5gstwgcmxycii3q7zi4jemjwhb7wienmlapx2ni6ja
type: modelVersionSet

## Register Model
Save the model to the model catalog

In [27]:
# Register the model
model_id = huggingface_pipeline_model.save(display_name="Healthcare NER Model",
                                            model_version_set=mvs,
                                            version_label="Version 1")
model_id

Model is successfully loaded.
['.model-ignore', 'special_tokens_map.json', 'tokenizer.json', 'config.json', 'runtime.yaml', 'score.py', 'test_json_output.json', 'model.safetensors', 'sentencepiece.bpe.model', 'tokenizer_config.json']


loop1:   0%|          | 0/4 [00:00<?, ?it/s]

'ocid1.datasciencemodel.oc1.ca-toronto-1.amaaaaaa3hvgr2qaxmjd5sf3kv3vri4rsgdo3hzw66bq6e4in65taft2y3yq'

## Deploy and Generate Endpoint
Deploy and create an endpoint for the huggingface_pipeline_model


In [28]:
# Deploy and create an endpoint for the huggingface_pipeline_model
huggingface_pipeline_model.deploy(
    display_name="Healthcare NER Mode Deployment", 
    description="Healthcare NER Mode Deployment",
    deployment_log_group_id="ocid1.loggroup.oc1.ca-toronto-1.amaaaaaa3hvgr2qafnnj4nyxd3bxxxmfx35bbmsmftedosa3fxin7ag46jia",
    deployment_access_log_id="ocid1.log.oc1.ca-toronto-1.amaaaaaa3hvgr2qah2mx7uxr5nl7rtqtaygsui3liy3uqyor5wp6zs7xsdua",
    deployment_predict_log_id="ocid1.log.oc1.ca-toronto-1.amaaaaaa3hvgr2qah2mx7uxr5nl7rtqtaygsui3liy3uqyor5wp6zs7xsdua"
)
print(f"Endpoint: {huggingface_pipeline_model.model_deployment.url}")


Model Deployment OCID: ocid1.datasciencemodeldeployment.oc1.ca-toronto-1.amaaaaaa3hvgr2qauukxaeb3pel6d575orfdxngmkhy4fsmbz3bezlvqvalq


Creating model deployment:   0%|          | [00:00<?, ?it/s]

Endpoint: https://modeldeployment.ca-toronto-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.ca-toronto-1.amaaaaaa3hvgr2qauukxaeb3pel6d575orfdxngmkhy4fsmbz3bezlvqvalq


In [29]:
print(huggingface_pipeline_model.model_deployment.display_name)
print(huggingface_pipeline_model.model_deployment.description)
print(huggingface_pipeline_model.model_deployment.time_created)

Healthcare NER Mode Deployment
Healthcare NER Mode Deployment
2024-08-30 01:49:54.888000+00:00


## Run Prediction against Endpoint
Generate prediction by invoking the deployed endpoint


In [30]:
# Generate prediction by invoking the deployed endpoint
preds = huggingface_pipeline_model.predict(data)

#print predictions
for pred in preds['prediction']:
    print(pred['word'],pred['entity_group'], pred['score'], pred['start'], pred['end'])

antibiotiques MedicationVaccine 0.742815375328064 21 34


Invoke model HTTP endpoint (Rest-based API) 

In [33]:
import requests
import oci
from oci.signer import Signer

# Get notebook session's resource principal
signer = oci.auth.signers.get_resource_principals_signer()


prediction_endpoint = f'{huggingface_pipeline_model.model_deployment.url}/predict'

print(f"Invoking Model Endpoint: {prediction_endpoint}")
print(f"Medical sample text : {data}")

body = {"inputs":data} # payload
headers = {} # headers

preds = requests.post(prediction_endpoint, json=body, auth=signer, headers=headers).json()

print(f"Extracted medical entities : {preds}")


Invoking Model Endpoint: https://modeldeployment.ca-toronto-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.ca-toronto-1.amaaaaaa3hvgr2qauukxaeb3pel6d575orfdxngmkhy4fsmbz3bezlvqvalq/predict
Medical sample text : Le medecin donne des antibiotiques en cas d'infections des voies respiratoires.
Extracted medical entities : {'prediction': [{'entity_group': 'MedicationVaccine', 'score': 0.742815375328064, 'word': 'antibiotiques', 'start': 21, 'end': 34}]}


Our Trained Model is now deployed and ready for inference.
This concludes our deployment notebook.